<a href="https://colab.research.google.com/github/mister-X-is-your-father/basic-website/blob/master/Kajabi%E3%82%BF%E3%82%B0%E4%BB%98%E3%81%91%E3%81%97%E3%81%9F%E3%82%89%E6%8B%9B%E5%BE%85%E3%83%A1%E3%83%BC%E3%83%AB%E9%80%81%E4%BF%A1Discord_Bot_JOIN%E6%A4%9C%E7%9F%A5%E8%AA%8D%E8%A8%BC%E6%A9%9F%E8%83%BD%E4%BB%98%E3%81%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install discord.py Flask requests python-dotenv sendgrid gunicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 46.0.1 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.1 which is incompatible.


In [ ]:
# --- ライブラリのインポート ---
import os
import threading
import requests
from flask import Flask, request, jsonify
import discord
from google.colab import userdata
from pyngrok import ngrok
import nest_asyncio
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import asyncio


nest_asyncio.apply()

# --- シークレットから情報を取得 ---
# Discord用
DISCORD_BOT_TOKEN = userdata.get('DISCORD_BOT_TOKEN')
DISCORD_GUILD_ID = int(userdata.get('DISCORD_GUILD_ID'))
DISCORD_MEMBER_ROLE_ID = int(userdata.get('DISCORD_MEMBER_ROLE_ID'))
DISCORD_WELCOME_CHANNEL_ID = int(userdata.get('DISCORD_WELCOME_CHANNEL_ID'))
# Kajabi用
KAJABI_CLIENT_ID = userdata.get('KAJABI_API_KEY')
KAJABI_CLIENT_SECRET = userdata.get('KAJABI_API_SECRET')
KAJABI_SITE_ID = userdata.get('KAJABI_SITE_ID')
# ngrok用
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
# SendGrid用
SENDGRID_API_KEY = userdata.get('SENDGRID_API_KEY')
FROM_EMAIL = 'noreply@oshaberi-lab.com'


# --- Flaskアプリケーションのセットアップ ---
app = Flask(__name__)

# --- Discord Botのセットアップ ---
intents = discord.Intents.default()
intents.members = True
intents.message_content = True
client = discord.Client(intents=intents)


# ===================================================================
# ★★★ Kajabi API 連携部分 ★★★
# ===================================================================

def get_kajabi_access_token():
    """KajabiのClient IDとClient Secretを使ってアクセストークンを取得する関数。"""
    token_url = "https://api.kajabi.com/v1/oauth/token"
    payload = {'client_id': KAJABI_CLIENT_ID, 'client_secret': KAJABI_CLIENT_SECRET, 'grant_type': 'client_credentials'}
    try:
        response = requests.post(token_url, data=payload)
        response.raise_for_status()
        return response.json().get("access_token")
    except requests.exceptions.RequestException as e:
        print(f"Kajabiアクセストークン取得エラー: {e}\n応答内容: {response.text}")
        return None

def verify_kajabi_customer(email_to_verify):
    """アクセストークンを取得し、Kajabiの顧客情報を照合する関数。"""
    access_token = get_kajabi_access_token()
    if not access_token: return False
    contacts_url = "https://api.kajabi.com/v1/contacts"
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'filter[site_id]': KAJABI_SITE_ID, 'filter[search]': email_to_verify}
    try:
        response = requests.get(contacts_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        if data.get("data") and len(data["data"]) > 0:
            if data["data"][0].get("attributes", {}).get("email", "").lower() == email_to_verify.lower():
                print(f"Kajabi照合成功: {email_to_verify}")
                return True
        print(f"Kajabi照合失敗: {email_to_verify} はKajabiに見つかりませんでした。")
        return False
    except requests.exceptions.RequestException as e:
        print(f"Kajabi顧客情報検索エラー: {e}\n応答内容: {response.text}")
        return False

# ===================================================================
# ★★★ WebhookとDiscordイベント処理 ★★★
# ===================================================================

async def create_single_use_invite(channel_id):
    """Discordのチャンネルに1回限りの招待コードを非同期で作成する関数"""
    try:
        await client.wait_until_ready()
        channel = client.get_channel(channel_id)
        if channel:
            invite = await channel.create_invite(max_age=86400, max_uses=1, unique=True)
            print(f"招待コード生成成功: {invite.url}")
            return invite.url
        else:
            print(f"エラー: 招待を作成するチャンネル(ID: {channel_id})が見つかりません。")
            return None
    except Exception as e:
        print(f"招待コード生成中にエラー: {e}")
        return None

def send_invite_email(email, name, discord_invite_url):
    """SendGridを使って招待メールを送信する関数"""
    message = Mail(
        from_email=(FROM_EMAIL, 'おしゃべりラボ'),
        to_emails=email,
        subject=f"【重要】コミュニティへのご招待",
        html_content=f"""
        <p>{name}様</p>
        <p>この度はコミュニティにご参加いただき、誠にありがとうございます。</p>
        <p>あなた専用の招待リンクをご用意しました。以下のリンクからDiscordサーバーにご参加ください。</p>
        <p><strong>このリンクは1度しか使えず、有効期限は24時間です。</strong></p>
        <p><a href="{discord_invite_url}">{discord_invite_url}</a></p>
        <p>サーバー参加後、ボットからDMでご案内が届きますので、案内に従って認証を完了してください。</p>
        <p>よろしくお願いいたします。</p>
        """
    )
    try:
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        response = sg.send(message)
        print(f"SendGridメール送信成功: To={email}, Status={response.status_code}")
        return True
    except Exception as e:
        print(f"SendGridメール送信エラー: {e}")
        return False

@app.route('/zapier-webhook', methods=['POST'])
def handle_webhook():
    data = request.json
    user_email = data.get('email')
    user_name = data.get('name')
    if not user_email or not user_name:
        return jsonify({"status": "error", "message": "email and name are required"}), 400

    print(f"ZapierからWebhook受信: {user_email}")
    future = asyncio.run_coroutine_threadsafe(create_single_use_invite(DISCORD_WELCOME_CHANNEL_ID), client.loop)
    invite_url = future.result()

    if not invite_url:
        print("エラー: 招待URLの取得に失敗したため、メール送信を中止します。")
        return jsonify({"status": "error", "message": "Failed to create Discord invite"}), 500

    success = send_invite_email(user_email, user_name, invite_url)
    if success:
        return jsonify({"status": "success", "message": "Invite email sent"}), 200
    else:
        return jsonify({"status": "error", "message": "Failed to send invite email"}), 500

@client.event
async def on_ready():
    print(f'{client.user} としてログインしました。')
    print('Botは正常に起動し、イベントを待機しています...')

@client.event
async def on_member_join(member):
    print(f'{member.name} がサーバーに参加しました。')
    try:
        await member.send("コミュニティへようこそ！\n認証を完了するために、あなたがKajabiに登録したメールアドレスを、このチャットに送信してください。")
        print(f"{member.name} にDMを送信しました。")
    except discord.errors.Forbidden:
        print(f"エラー: {member.name} にDMを送信できませんでした。")

@client.event
async def on_message(message):
    if message.author == client.user: return
    if isinstance(message.channel, discord.DMChannel):
        email_to_verify = message.content.strip()
        print(f"{message.author.name} からDMを受信: {email_to_verify}")

        # Kajabiの照合を実行（ブロッキング処理なので、Botを停止させないよう別スレッドで実行）
        is_verified = await client.loop.run_in_executor(
            None, verify_kajabi_customer, email_to_verify
        )

        if is_verified:
            try:
                guild = client.get_guild(DISCORD_GUILD_ID)
                if not guild:
                    print(f"エラー: サーバー(ID: {DISCORD_GUILD_ID})が見つかりません。")
                    return
                member = await guild.fetch_member(message.author.id)
                role = guild.get_role(DISCORD_MEMBER_ROLE_ID)
                if member and role:
                    await member.add_roles(role)
                    await message.channel.send("認証が完了しました！サーバーの全てのチャンネルが利用可能になりました。")
                    print(f"{member.name} にロールを付与しました。")
                else:
                    await message.channel.send("エラー：サーバー内であなた、またはロールを見つけられませんでした。")
            except Exception as e:
                await message.channel.send("エラーが発生しました。運営者にお問い合わせください。")
                print(f"ロール付与中にエラー: {e}")
        else:
            await message.channel.send("ご入力のメールアドレスでは登録が確認できませんでした。もう一度、正しいメールアドレスを送信してください。")

# --- Flaskとngrokを起動 ---
port = 5000
ngrok.set_auth_token(NGROK_AUTHTOKEN)
public_url = ngrok.connect(port)
print(f"Zapierに設定するWebhook URL: {public_url}/zapier-webhook")

threading.Thread(target=lambda: app.run(host='0.0.0.0', port=port, use_reloader=False)).start()

# --- Discord Botを起動 ---
client.run(DISCORD_BOT_TOKEN)


2025-09-25 10:20:54 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token


Zapierに設定するWebhook URL: NgrokTunnel: "https://043b811a7f2d.ngrok-free.app" -> "http://localhost:5000"/zapier-webhook
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
2025-09-25 10:20:56 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: a274a9ff87c4e186174e80ea354918bd).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: a274a9ff87c4e186174e80ea354918bd).


user_manage_bot;#0750 としてログインしました。
Botは正常に起動し、イベントを待機しています...
ZapierからWebhook受信: ikimonogakaru@gmail.com
招待コード生成成功: https://discord.gg/DsC5mYgp


INFO:werkzeug:127.0.0.1 - - [25/Sep/2025 10:23:16] "POST /zapier-webhook HTTP/1.1" 200 -


SendGridメール送信成功: To=ikimonogakaru@gmail.com, Status=202
dentakun_73480 がサーバーに参加しました。
dentakun_73480 にDMを送信しました。
dentakun_73480 からDMを受信: oij
Kajabi照合失敗: oij はKajabiに見つかりませんでした。
